# 합성곱 연산

완전연결 층과 합성공 층 사이의 차이

Dense 층은 입력 특성 공간에 있는 전역 패턴을 학습.
합성곱 층은 지역패턴을 학습.

컨브넷의 성질 
 - 학습된 패턴은 평행 이동 불변성을 가진다.  
     일반화 하는 능력
     
 - 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다.  
     이전층에서 다음층으로 넘어갈 때 이전 층의 특성으로 구성된 더큰 패턴을 학습.
 
 합성곱 연산  
 특성 맵이라고 부르는 3D 텐서를 적용. (높이와 넓이)(채널) 로 이루어져 있음.
 ex) RGB  이미지 : 가로 * 세로 * 3채널(R,G,B)
 ex) 흑백 이미지 : 가로 * 세로 * 1채널(회색톤)
 
 요약
 합성곱은 연산은 **입력** 특성맵에서 작은 패치들을 추출하고 이 모든 패치에 같은 변환을 적용하여 **출력** 특성맵을 만든다.
 
 
     


출력 특성 맵도 3D 텐서이다.
기존의 RGB 3채널이 더 이상 R,G,B를 의미하진 않는다. 일종의 필터를 의미한다. 
ex)
MNIST 예제에서 보면 (28,28,1)을 입력받아 (26,26,32)를 출력함.  
즉, 이것은 입력에 대해 32개의 필터를 적용하는 것이다. 이것들은 각각 (26, 26) 크기의 배열값으로 이루어져 있다.





## 합성곱의 파라미터

- 입력으로부터 뽑아낼 패치의 크기:  
    일반적으로 3*3 혹은 5*5 크기의 패치를 사용.
- 특성 맵의 출력 깊이:
    합성곱으로 계산할 필터의 수.


패치의 크기만큼 **슬라이딩**하면서 모든위치에서의 **3D 특성패치**를 추출하는 방식으로 합성곱이 작동합니다.  
공간상 위치는 입력 특성맵의 위치를 따라갑니다.

출력 특성 맵의 높이와 넓이는 입력 특성맵의 높이와 넓이와는 다를 수 있다.
이유:
    - 경계문제. 입력 특성 맵에 패딩을 추가하여 대응할 수 있다.  
    - 스트라이드 사용 여부에 따라 다르다.  



### 경계 문제와 패딩 이해하기

예를들어 5*5의 입력값에 3*3 크기의 패치를 적용하면 2개의 타일이 줄어든다.

앞의 예제에서 28*28 크기의 입력이 26*26 크기로 줄어들었다.  

입력과 동일한 높이와 넓이를 가진 출력을 얻고 싶으면 **패딩**을 사용하여 얻을 수 있다.

케라스의 Conv2D 층에선 패딩 매개변수 설정이 가능하다.

- "valid" : 패딩 미사용
- "same"  : 패딩 사용 (입력과 동일한 높이와 넓이를 가진 출력을 만들 수 있게 패딩)



### 합성곱 스트라이드 이해하기  

스트라이드의 기본값은 "1"
보통 스트라이드1 스트라이드2 라고 부른다.
스트라이드 2는 특성 맵의 넓이와 높이가 2의 배수로 다운 샘플링 되었다는 뜻.

컨브넷 파라미터 설정은 "딥러닝을 위한 콘볼루션 계산 가이드"를 [참고](https://goo.gl/qvNTyu).



# 최대 풀링 연산  

앞의 예제를 보면 MaxPooling2D 층마다 특성맵의 크기가 절반으로 줄어든것을 볼 수 있다.
ex) 26*26 -> 13*13
  
최대 풀링은 2*2 윈도우와 스트라이드 2를 사용하여 특성맵을 **절반 크기**로 다운샘플링 하는것.
합성곱은 3*3 윈도우와 스트라이드 1을 사용.  



## 풀링 층을 사용하지 않으면....  

풀링층을 쓰지않는 모델 구조.

In [3]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### 이 구조의 문제점
1. 특성의 공간적 계층 구조를 학습하는데 도움이 되지 않는다.
    고수준의 패턴은 정보가 적어 학습하기에 충분치 않다.
2. 최종 특성 맵은 22 * 22 * 64 = 30,976개의 가중치로 아주 많다.
    작은 모델치곤 가중치가 너무 많아 과대적합이 발생.
    
다운샘플링의 이유는 가중치 개수를 줄이기 위해서!
공간적 계층구조를 구성하기 위함!


최대 풀링 대신 **평균 풀링**을 사용할 수도 있음.  

최대 풀링이 다른 방법들 보다 더 잘 작동하는 편이므로 최대풀링을 사용하는것을 권장.(데이터에 따라 다름..)  


소비의 유형

그 물건이 없어서   (need)
그 물건이 망가져서 (broke)
더 좋아보여서 (better)
그냥    (no reason)